In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image, ImageOps # Necesitas: pip install pillow  # Cambia esto por la ruta de tu imagen

def predecir_ropa(ruta_imagen):
        # 1. CARGAR EL MODELO
        # Cargamos el cerebro que ya entrenaste
        model = tf.keras.models.load_model('mi_modelo.keras')

        # 2. CARGAR Y PRE-PROCESAR LA IMAGEN (La parte crítica)
        # A. Abrir imagen como escala de grises ('L')
        img = Image.open(ruta_imagen).convert('L')

        # B. INVERTIR COLORES
        # Si el fondo es blanco, lo volvemos negro para que coincida con el entrenamiento
        img = ImageOps.invert(img)

        # C. REDIMENSIONAR
        # El modelo es miope, solo ve 28x28 píxeles.
        img = img.resize((28, 28))

        # D. CONVERTIR A NÚMEROS
        # De imagen a matriz NumPy y normalizar (0 a 1)
        img_array = np.array(img) / 255.0
        
        # E. DAR FORMA DE LOTE (BATCH)
        # El modelo espera una lista de fotos, no una sola.
        # Transformamos (28, 28) -> (1, 28, 28, 1)
        img_array = img_array.reshape(1, 28, 28, 1)

        # 3. PREDICCIÓN
        prediction = model.predict(img_array)

        # 4. INTERPRETAR RESULTADO
        clases = ['Camiseta', 'Pantalón', 'Jersey', 'Vestido', 'Abrigo',
                'Sandalia', 'Camisa', 'Zapatilla', 'Bolso', 'Bota']
        
        indice_ganador = np.argmax(prediction) # ¿Cuál neurona gritó más fuerte?
        confianza = np.max(prediction)         # ¿Qué tan fuerte gritó? (0-1)

        print(f"------------------------------------------------")
        print(f"Imagen: {ruta_imagen}")
        print(f"Predicción: {clases[indice_ganador].upper()}")
        print(f"Confianza: {confianza*100:.2f}%")
        print(f"------------------------------------------------")

In [1]:
predecir_ropa(Machine_Learning_Projects)

NameError: name 'predecir_ropa' is not defined

**Dropout (Apagar Neuronas)**

In [ ]:
model = keras.Sequential([
    # CAPA 1
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    # CAPA 2
    keras.layers.Conv2D(64, (3, 3), activation='relu'),